In [28]:
# Import Libraries

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, LSTMCell
import yfinance as yf
import pandas as pd
from datetime import datetime

In [54]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to train LSTM model
def train_lstm_model(symbol):
    # Download historical data
    data = yf.download(symbol, start='2020-01-01', end='2023-12-31')
    
    # Extract 'Close' prices
    close_prices = data['Close'].values.reshape(-1, 1)

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)

    # Create training dataset
    x_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape for LSTM model
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(RNN(LSTMCell(128), return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(RNN(LSTMCell(64), return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=2)

    return model, scaler

# Train models for multiple stocks
stock_symbols = ["AAPL", "GOOGL"]
trained_models = {}

for symbol in stock_symbols:
    model, scaler = train_lstm_model(symbol)
    trained_models[symbol] = {"model": model, "scaler": scaler}


[*********************100%%**********************]  1 of 1 completed
Epoch 1/2
940/940 [==============================] - 122s 129ms/step - loss: 0.0044
Epoch 2/2
940/940 [==============================] - 121s 129ms/step - loss: 0.0015
[*********************100%%**********************]  1 of 1 completed
Epoch 1/2
940/940 [==============================] - 122s 129ms/step - loss: 0.0046
Epoch 2/2
940/940 [==============================] - 121s 128ms/step - loss: 0.0020


In [63]:
# User selects a stock
selected_stock = "AAPL"

# Download recent data for prediction
data_for_prediction = yf.download(selected_stock, start='2023-12-01', end='2023-12-20')
input_data = data_for_prediction['Close'].values.reshape(-1, 1)

# Scale input data using the corresponding scaler
scaler_for_prediction = trained_models[selected_stock]["scaler"]
scaled_input_data = scaler_for_prediction.transform(input_data)

# Reshape for LSTM model input
num_time_steps = min(len(data_for_prediction), 60)
scaled_input_data = np.reshape(scaled_input_data, (1, num_time_steps, 1))

# Make predictions using the corresponding model
model_for_prediction = trained_models[selected_stock]["model"]
predicted_scaled_price = model_for_prediction.predict(scaled_input_data)

# Inverse transform to get the predicted price in the original scale
predicted_price = scaler_for_prediction.inverse_transform(predicted_scaled_price)

# Print the predicted price
print(f"Predicted Closing Price for {selected_stock}: {predicted_price[0][0]}")


[*********************100%%**********************]  1 of 1 completed

ValueError: in user code:

    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\koh46\anaconda3\envs\tfdml_plugin\lib\site-packages\keras\e